In [6]:
from ClosestNeighbor import ClosestNeighbor
from NeighborsResearch import NeighborsResearch
from VND import VND

from tqdm import tqdm
from datetime import datetime
import pandas as pd
import statistics

# Construção da Tabela

## Constantes

In [7]:
f = open("instancias_teste/otimos", "r")
otimos = {}
for x in f:
  line = x.split(" ")
  otimos[line[0]] = int(line[2])

execucoes = 10
vnd = VND()
tabela = pd.DataFrame(index = otimos.keys(), columns = ["otimo", "HC - Média solução", 
"HC - Melhor Solução", "HC - Média Tempo", "HC - gap", "(HC + VND) - Média solução", 
"(HC + VND) - Melhor Solução", "(HC + VND) - Média Tempo", "(HC + VND) - gap"])

## Execução do algoritmo e preenchimento da tabela

In [8]:
for situation in tqdm(otimos):
    tabela["otimo"][situation] = otimos[situation]
    solucoes_time_HC = []
    solucoes_time_HC_VND = []
    solucoes_HC = []
    solucoes_HC_VND = []
    for i in range(execucoes):
        HeuristicAlgorithm = ClosestNeighbor(situation)
        now_HC_VND = datetime.now()
        now_HC = datetime.now()
        HeuristicAlgorithm.closestNeighbor()
        later_HC = datetime.now()
        solucoes_time_HC.append((later_HC - now_HC).total_seconds())
        
        paths = HeuristicAlgorithm.get_truck_path()
        total_distances = HeuristicAlgorithm.get_truck_total_distance()
        matrix = HeuristicAlgorithm.get_distance_matrix()
        solucoes_HC.append(sum(total_distances.values()))
        
        new_paths, new_total_distances = vnd.run_vnd(paths, total_distances, matrix)
        later_HC_VND = datetime.now()
        solucoes_HC_VND.append(sum(new_total_distances.values()))
        solucoes_time_HC_VND.append((later_HC_VND - now_HC_VND).total_seconds())
        
    tabela["HC - Média Tempo"][situation] = statistics.mean(solucoes_time_HC)
    tabela["HC - Média solução"][situation] = statistics.mean(solucoes_HC)
    tabela["HC - Melhor Solução"][situation] = sorted(solucoes_HC)[0]
    tabela["HC - gap"][situation] = ((tabela["HC - Melhor Solução"][situation] - otimos[situation]) / otimos[situation]) * 100

    tabela["(HC + VND) - Média Tempo"][situation] = statistics.mean(solucoes_time_HC_VND)
    tabela["(HC + VND) - Média solução"][situation] = statistics.mean(solucoes_HC_VND)
    tabela["(HC + VND) - Melhor Solução"][situation] = sorted(solucoes_HC_VND)[0]
    tabela["(HC + VND) - gap"][situation] = ((tabela["(HC + VND) - Melhor Solução"][situation] - otimos[situation]) / otimos[situation]) * 100

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 21.33it/s]


In [9]:
tabela

,otimo,HC - Média solução,HC - Melhor Solução,HC - Média Tempo,HC - gap,(HC + VND) - Média solução,(HC + VND) - Melhor Solução,(HC + VND) - Média Tempo,(HC + VND) - gap
P-n16-k8,450,536.9,511,0.0006,13.5556,525.9,510,0.000899,13.3333
P-n19-k2,212,410.8,367,0.0003997,73.1132,260.2,243,0.0020958,14.6226
P-n20-k2,216,373,321,0.0004976,48.6111,255.5,231,0.0021994,6.94444
P-n23-k8,529,720.7,674,0.0001003,27.4102,694.7,656,0.0001975,24.0076
P-n55-k7,568,1147.1,1004,0.0024995,76.7606,917,793,0.0075103,39.6127
P-n51-k10,741,1329.5,1235,0.0027075,66.6667,1163.7,1057,0.0049016,42.6451
P-n50-k10,696,1230.2,1075,0.002102,54.454,1069.2,929,0.0040003,33.477
P-n45-k5,510,1044,920,0.0015005,80.3922,799.9,732,0.0068953,43.5294
